In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
import pickle

file_path = 'C:\\Users\\Akash\\Desktop\\Traffic Analysis and prediction\\traffic.csv'

df = pd.read_csv(file_path)
print(df)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
print(df.info())
print(df.describe())

In [ ]:
df = df.dropna()

In [ ]:
df['DateTime'] = pd.to_datetime(df['DateTime'])

In [ ]:
df['hour'] = df['DateTime'].dt.hour
df['day_of_week'] = df['DateTime'].dt.dayofweek
df['month'] = df['DateTime'].dt.month
df['year'] = df['DateTime'].dt.year



In [ ]:
print(df.info())
print(df.describe())

In [ ]:
# df_top20 = df.head(20)
# print(df_top20)

for column in df.columns:
    unique_values = df[column].unique()
    print(f"Unique values in column '{column}': {unique_values}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x='hour', y='Vehicles', hue='Junction',palette=['red', 'blue', 'green', 'yellow'])
plt.title('Traffic Volume by Hour')
plt.xlabel('Hour of the Day')
plt.ylabel('Number of Vehicles')
plt.legend(title='Junction')
plt.xticks(ticks=range(0, 24))
plt.show()



In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x='day_of_week', y='Vehicles', hue='Junction', palette=['red', 'blue', 'green', 'yellow'])
plt.title('Traffic Volume by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Number of Vehicles')
plt.legend(title='Junction')
plt.xticks(ticks=range(0, 7), labels=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday'])
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x='month', y='Vehicles', hue='Junction', palette=['red', 'blue', 'green', 'yellow'])
plt.title('Traffic Volume by Year')
plt.xlabel('Month of the Year')
plt.ylabel('Number of Occurrences')
plt.legend(title='Junction')
plt.xticks(ticks= df['month'].unique() )
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x='year', y='Vehicles', hue='Junction', palette=['red', 'blue', 'green', 'yellow'])
plt.title('Traffic Volume by Year')
plt.xlabel('Years')
plt.ylabel('Number of Occurrences')
plt.legend(title='Junction')
plt.xticks(ticks= df['year'].unique() )
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

features = df[['hour', 'day_of_week', 'month', 'year', 'Junction']]
target = df['Vehicles']

features = pd.get_dummies(features, columns=['Junction'], drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
print('Mean Absolute Error:', mean_absolute_error(y_test, predictions))


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_test.values[:100], label='Actual')
plt.plot(predictions[:100], label='Predicted')
plt.title('Actual vs Predicted Traffic Volume')
plt.xlabel('Sample')
plt.ylabel('Number of Vehicles')
plt.legend()
plt.show()


In [ ]:
def categorize_traffic(volume):
    if volume < 20:
        return "No traffic"
    elif volume < 50:
        return "Less traffic"
    elif volume < 100:
        return "Heavy traffic"
    else:
        return "Worst case"

In [ ]:
def predict_traffic(date_str, time_str, junction, model, feature_columns):
    # Combine date and time into a single datetime object
    date_time_str = f"{date_str} {time_str}"
    date_time = pd.to_datetime(date_time_str)

    # Extract features from the datetime object
    hour = date_time.hour
    day_of_week = date_time.dayofweek
    month = date_time.month
    year = date_time.year

    # Prepare the input data for the model
    input_data = pd.DataFrame({
        'hour': [hour],
        'day_of_week': [day_of_week],
        'month': [month],
        'year': [year]
    })

    # One-hot encode the junction feature
    junction_dummies = pd.get_dummies([junction], prefix='Junction')

    # Ensure that the dummy variables match the training columns
    for col in feature_columns:
        if col.startswith('Junction') and col not in junction_dummies.columns:
            junction_dummies[col] = 0

    # Combine the features
    input_data = pd.concat([input_data, junction_dummies], axis=1)

    # Ensure the input data has the same columns as the training data
    for col in feature_columns:
        if col not in input_data.columns:
            input_data[col] = 0

    # Arrange the columns in the same order as during training
    input_data = input_data[feature_columns]

    # Predict traffic volume
    predicted_volume = model.predict(input_data)[0]

    # Categorize traffic
    traffic_category = categorize_traffic(predicted_volume)

    return traffic_category


In [ ]:
feature_columns = X_train.columns

In [ ]:
# Example usage
date_str = input("Enter date (YYYY-MM-DD): ")  # Separate date input
time_str = input("Enter time (HH:MM:SS): ")   # Separate time input
junction = int(input("Enter junction (integer): "))  # Junction as an integer

# Predict traffic
traffic_category = predict_traffic(date_str, time_str, junction, model, feature_columns)
print(f'Traffic prediction for {date_str} {time_str} at junction {junction}: {traffic_category}')


In [ ]:
average_traffic = df.groupby('Junction')['Vehicles'].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x='Junction', y='average_traffic')
plt.title('Traffic Volume by Year')
plt.xlabel('Years')
plt.ylabel('Number of Occurrences')
plt.legend(title='Junction')
plt.xticks(ticks= df['year'].unique() )
plt.show()